# Install dependencies


In [1]:
!pip install -qU torch==2.4.0 torchvision==0.19.0 "ray[client,train]"==2.34.0

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"Train Epoch {epoch}")
        for X, y in train_dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            print(f"Test Epoch {epoch}")
            for X, y in test_dataloader:
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [4]:
ray.init(
    address="ray://raycluster-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.4.0", "torchvision==0.19.0"]},
)

SIGTERM handler is not set because current thread is not the main thread.
2024-08-21 02:16:15,236	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.34.0
    Python: 3.9.19
This process on Ray Client was started with:
    Ray: 2.34.0
    Python: 3.9.13



Python version:,3.9.19
Ray version:,2.34.0
Dashboard:,http://10.0.3.163:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote(num_workers=7, use_gpu=False))

(autoscaler +4s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +4s) Adding 1 node(s) of type small-group.
(autoscaler +4s) Resized to 7 CPUs.
(train_fashion_mnist pid=408, ip=10.0.2.19) 
(train_fashion_mnist pid=408, ip=10.0.2.19) View detailed results here: /home/ray/ray_results/TorchTrainer_2024-08-20_19-17-58
(train_fashion_mnist pid=408, ip=10.0.2.19) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-08-20_19-13-24_762089_1/artifacts/2024-08-20_19-17-58/TorchTrainer_2024-08-20_19-17-58/driver_artifacts`
(autoscaler +1m46s) Adding 1 node(s) of type small-group.
(autoscaler +1m46s) Resized to 14 CPUs.
(autoscaler +1m46s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +1m51s) No available node types can fulfill resource requ

(train_fashion_mnist pid=408, ip=10.0.2.19) Ignore this message if the cluster is autoscaling. Training has not started in the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 8.0 CPUs and 0 GPUs, but the cluster only has 7.0 CPUs and 0 GPUs available. Stop the training and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


(autoscaler +2m47s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +2m53s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +2m58s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m3s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +3m8s) No available node types can fulfill resource requests {'bundle_group_e892fc3524f8dc4017035d6d0aab02000000': 0.001}*1. Add suitable node types to this cluster to resolve this issu

(RayTrainWorker pid=470, ip=10.0.2.35) Setting up process group for: env:// [rank=0, world_size=7]
(TorchTrainer pid=424, ip=10.0.2.35) Started distributed worker processes: 
(TorchTrainer pid=424, ip=10.0.2.35) - (node_id=bf8b5e9e63623249cba282fa3469c5db6f49cb8d6e42b7f6b017d38b, ip=10.0.2.35, pid=470) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=424, ip=10.0.2.35) - (node_id=bf8b5e9e63623249cba282fa3469c5db6f49cb8d6e42b7f6b017d38b, ip=10.0.2.35, pid=471) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=424, ip=10.0.2.35) - (node_id=bf8b5e9e63623249cba282fa3469c5db6f49cb8d6e42b7f6b017d38b, ip=10.0.2.35, pid=472) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=424, ip=10.0.2.35) - (node_id=bf8b5e9e63623249cba282fa3469c5db6f49cb8d6e42b7f6b017d38b, ip=10.0.2.35, pid=473) world_rank=3, local_rank=3, node_rank=0
(TorchTrainer pid=424, ip=10.0.2.35) - (node_id=bf8b5e9e63623249cba282fa3469c5db6f49cb8d6e42b7f6b017d38b, ip=10.0.2.35, pid=474) world_rank=4, local_

(RayTrainWorker pid=1087, ip=10.0.2.19) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=471, ip=10.0.2.35) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=471, ip=10.0.2.35) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz
(RayTrainWorker pid=1087, ip=10.0.2.19) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


...

(RayTrainWorker pid=471, ip=10.0.2.35) Extracting /home/ray/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=471, ip=10.0.2.35) 
(RayTrainWorker pid=471, ip=10.0.2.35) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=1087, ip=10.0.2.19) Extracting /home/ray/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=1087, ip=10.0.2.19) 
(RayTrainWorker pid=1087, ip=10.0.2.19) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz


100%|██████████| 4422102/4422102 [00:01<00:00, 3515113.43it/s]


(RayTrainWorker pid=471, ip=10.0.2.35) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=1087, ip=10.0.2.19) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=471, ip=10.0.2.35) Extracting /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=471, ip=10.0.2.35) 
(RayTrainWorker pid=1087, ip=10.0.2.19) Extracting /home/ray/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ray/data/FashionMNIST/raw
(RayTrainWorker pid=1087, ip=10.0.2.19) 


100%|██████████| 5148/5148 [00:00<00:00, 37881187.71it/s]
(RayTrainWorker pid=1087, ip=10.0.2.19) Moving model to device: cpu
(RayTrainWorker pid=1087, ip=10.0.2.19) Wrapping provided model in DistributedDataParallel.
(RayTrainWorker pid=470, ip=10.0.2.35) Moving model to device: cpu
(RayTrainWorker pid=470, ip=10.0.2.35) Wrapping provided model in DistributedDataParallel.


(RayTrainWorker pid=1087, ip=10.0.2.19) Train Epoch 0
(RayTrainWorker pid=475, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=473, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=470, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=471, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=474, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=472, ip=10.0.2.35) Train Epoch 0
(RayTrainWorker pid=1087, ip=10.0.2.19) Test Epoch 0
(RayTrainWorker pid=475, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=473, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=470, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=471, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=474, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=472, ip=10.0.2.35) Test Epoch 0
(RayTrainWorker pid=471, ip=10.0.2.35) Train Epoch 1
(RayTrainWorker pid=1087, ip=10.0.2.19) Train Epoch 1
(RayTrainWorker pid=470, ip=10.0.2.35) Train Epoch 1
(RayTrainWorker pid=474, ip=10.0.2.35) Train Epoch 1
(RayTrainWorker pid=475, ip=10.0.2.35) Train Epoch

# Shut down Ray workers


In [6]:
ray.shutdown()

(train_fashion_mnist pid=408, ip=10.0.2.19) 
(train_fashion_mnist pid=408, ip=10.0.2.19) Training completed after 10 iterations at 2024-08-20 19:27:21. Total running time: 9min 22s
(train_fashion_mnist pid=408, ip=10.0.2.19) 
(train_fashion_mnist pid=408, ip=10.0.2.19) Training result: Result(
(train_fashion_mnist pid=408, ip=10.0.2.19)   metrics={'loss': 0.359105228656248, 'accuracy': 0.8768369489153254},
(train_fashion_mnist pid=408, ip=10.0.2.19)   path='/home/ray/ray_results/TorchTrainer_2024-08-20_19-17-58/TorchTrainer_95938_00000_0_2024-08-20_19-17-58',
(train_fashion_mnist pid=408, ip=10.0.2.19)   filesystem='local',
(train_fashion_mnist pid=408, ip=10.0.2.19)   checkpoint=None
(train_fashion_mnist pid=408, ip=10.0.2.19) )


(train_fashion_mnist pid=408, ip=10.0.2.19) Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/TorchTrainer_2024-08-20_19-17-58' in 0.0027s.
